In [3]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np

np.seterr(all='ignore')
rcParams['figure.figsize'] = (14, 6)

#扩展MOD
from funcat import * #FUNCAT的同花顺/通达信指标模块：和：&，或：|，其他基本相同。可以使用指标选股！！！
#from pytdx import * #通达信数据更新模块到QUANTAXIS的mongoDB
from tushare import * #TUSHARE数据连接更新。
from rqalpha import * #RQALPHA本地数据连接，通过 rqalpha update_bundle 8点后更新全部数据库

In [5]:
import datetime
import numpy as np
import pandas as pd
import talib

#问题是如何拿到每周数据 price的
def init(context):
    context.stocks = index_components('000300.XSHG')
    scheduler.run_daily(sell,time_rule=market_close(minute = 20))
    scheduler.run_daily(buy,time_rule = market_open(hour =1))

def stocks_list(context,bar_dict):
    buy_list= []
    for stock in context.stocks:
        hist_high = history_bars(stock,30,'1d',fields='high')
        hist_low = history_bars(stock,30,'1d',fields='low')
        hist_close = history_bars(stock,30,'1d',fields='close')
        hist_volume = history_bars(stock,30,'1d',fields='volume')
        
        #求周线的kd金叉
        hist_m_high = history_bars(stock,150,'1d',fields='high')[::5]
        hist_m_low = history_bars(stock,150,'1d',fields='low')[::5]
        hist_m_close = history_bars(stock,150,'1d',fields='close')[::5]
        
        
        slowk,slowd = talib.STOCH(hist_high,hist_low,hist_close,fastk_period = 9,slowk_period = 3,slowk_matype=0,slowd_period=3,slowd_matype=0)
        
        slowk_m,slowd_m = talib.STOCH(hist_m_high,hist_m_low,hist_m_close,fastk_period = 9,slowk_period = 3,slowk_matype=0,slowd_period=3,slowd_matype=0)
        
        
        #k和d在小于35的地方金叉了
        if slowk[-1]<35 or slowd[-1]<35:
            if slowk[-1]>slowd[-1] and slowk[-2]<slowd[-2]: #金叉了
                if hist_volume[-1]>hist_volume[-2]: #量比增加了
                    if slowk_m[-1]<40:
                        if slowk_m[-1]-slowd_m[-1]>6:
                            print('day high',stock,hist_high,hist_low,hist_close)
                            print ('月high',stock,hist_m_high,hist_m_low,hist_m_close)
                            print ('buy',stock,slowk_m)
                            buy_list.append(stock)
    print('st to buy',len(buy_list))
    return buy_list

def sell(context,bar_dict):
    for stock in context.portfolio.positions.keys():
        hist_high = history_bars(stock,150,'1d',fields='high')[::5]
        hist_low = history_bars(stock,150,'1d',fields='low')[::5]
        hist_close = history_bars(stock,150,'1d',fields='close')[::5]
        slowk,slowd = talib.STOCH(hist_high,hist_low,hist_close,fastk_period = 9,slowk_period = 3,slowk_matype=0,slowd_period=3,slowd_matype=0)
        if slowk[-1]<slowd[-1]:
            print('卖出',stock,slowk)
            order_target_percent(stock,0,style = LimitOrder(bar_dict[stock].limit_down))
            
def buy(context,bar_dict):
    stock_to_buy = stocks_list(context,bar_dict)
    cash = context.portfolio.cash
    if len(stock_to_buy)>0:
        weight = 0.98*cash/len(stock_to_buy)
        for i in stock_to_buy:
            try:
                order_target_value(i,weight,style = LimitOrder(bar_dict[i].limit_up))
            except:
                order_target_value(i,weight)
    
    
    
    
    
    
    

